# Severstal Steel Challenge

This notebook is a submission for a Kaggle challenge hosted by Severstal Steel.  

https://www.kaggle.com/c/severstal-steel-defect-detection

The purpose is to build a segmentation model to label four types of manufacturing defects in images of steel.  I used the segmentation_models library to create UNet models with a pre determined backbone initialized with ImageNet weights.  For example, one can choose a ResNet34 backbone. 

https://github.com/qubvel/segmentation_models

My best model used an ensemble of ResNet34, Inceptionv3, and DenseNet121. Performance is measured with a Dice Coefficient.  My highest score was 0.84.  The winning submission was 0.91. 

Please read my blog if you would like to learn more about my ideas and motivation for this challenge.

https://www.funwithdatascience.com/2020/04/06/model-for-detecting-steel-defects-in-images/

In [1]:
#RLE functions from https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode/code

from skimage.io import imread, imshow, imread_collection, concatenate_images
import matplotlib.pyplot as plt
import os
import sys
import random
import warnings

import numpy as np
from numpy import fliplr, flipud
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

import segmentation_models


def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


Using TensorFlow backend.
C:\Users\Ashley\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ashley\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ashley\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ashley\AppData

Segmentation Models: using `keras` framework.


## Important Operating Parameters

In [18]:
IMG_HEIGHT = 256
IMG_WIDTH = 1600
IMG_CHANNELS = 3
DEFECT_CLASSES = 4
SCALE_FACTOR = 2
SAMPLE_SIZE = 950

train = pd.read_csv(r'C:\Users\Ashley\SeverstalSteel\train.csv')
IMAGE_PATH = r"C:\Users\Ashley\SeverstalSteel\train_images\\"
TEST_IMAGE_PATH = r"C:\Users\Ashley\SeverstalSteel\test_images\\"

## Load the Y Data

In [3]:
#import images
from skimage import color
from skimage import io

#import masks
class_indices = [[],[],[],[]]
multiclass_image_list = []

for ind in range(train.shape[0]):
    #Check if multiple masks exist, if so, add to a separate list of images
    if train['EncodedPixels'][ind] != '':
        current_file_name = train['ImageId'][ind]
        if train[train['ImageId'] == current_file_name].shape[0] == 1:
            class_indices[train['ClassId'][ind]-1].append(ind)
        elif current_file_name not in multiclass_image_list:
            multiclass_image_list.append(current_file_name)
            
            

    
    
Y_train = np.zeros((SAMPLE_SIZE*4, int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), int(DEFECT_CLASSES)), dtype=np.bool)


#get a random sample from each class
sample_set = []
for defect_type in range(4):
    partial_sample_set = np.random.permutation(int(SAMPLE_SIZE/5))
    for sample_index in partial_sample_set:
        sample_set.append(class_indices[defect_type][sample_index])

#Populate training data for the single class images
n=0        
for ind in range(len(sample_set)):

    img = rle2mask(train['EncodedPixels'][sample_set[ind]])
    img = resize(img, (int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR)), mode='constant', preserve_range=True)
    Y_train[n,:,:,(train['ClassId'][sample_set[ind]]-1)] = img
    Y_train[n+1,:,:,(train['ClassId'][sample_set[ind]]-1)] = fliplr(img)
    Y_train[n+2,:,:,(train['ClassId'][sample_set[ind]]-1)] = flipud(img)
    Y_train[n+3,:,:,(train['ClassId'][sample_set[ind]]-1)] = fliplr(flipud(img))
    n+=4
    
#Handle multiple mask instances
multiclass_samples = []
partial_sample_set_indices = np.random.permutation(int(SAMPLE_SIZE/5))
for ind in partial_sample_set_indices:
    multiclass_samples.append(multiclass_image_list[ind])
for current_file_name in multiclass_samples:
    #make temporary dataframe corresponding to this image
    df = train[train['ImageId'] == current_file_name]
    df = df.reset_index(drop=True)
    for ind in range(df.shape[0]):
        #add each mask to an entry
        img = rle2mask(df['EncodedPixels'][ind])
        img = resize(img, (int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR)), mode='constant', preserve_range=True)
        Y_train[n,:,:,(df['ClassId'][ind]-1)] = img
        Y_train[n+1,:,:,(df['ClassId'][ind]-1)] = fliplr(img)
        Y_train[n+2,:,:,(df['ClassId'][ind]-1)] = flipud(img)
        Y_train[n+3,:,:,(df['ClassId'][ind]-1)] = fliplr(flipud(img))
    n += 4

## Load the X Data

In [4]:
X_train = np.zeros((SAMPLE_SIZE*4, int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS), dtype=np.uint8)
#Populate X_train for single class images
n = 0
for ind in range(len(sample_set)):
    #img = color.rgb2gray(io.imread(IMAGE_PATH + train['ImageId'][ind]))
    img = imread(IMAGE_PATH + train['ImageId'][sample_set[ind]])
    img = resize(img, (int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS), mode='constant', preserve_range=True)
    X_train[n,:,:,:] = img
    X_train[n+1,:,:,:] = fliplr(img)
    X_train[n+2,:,:,:] = flipud(img)
    X_train[n+3,:,:,:] = fliplr(flipud(img))
    n += 4
    
#Populate X_train for multiclass images
for file_name in multiclass_samples:
    img = imread(IMAGE_PATH + file_name)
    #img = color.rgb2gray(io.imread(IMAGE_PATH + file_name))
    img = resize(img, (int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS), mode='constant', preserve_range=True)
    X_train[n,:,:,:] = img
    X_train[n+1,:,:,:] = fliplr(img)
    X_train[n+2,:,:,:] = flipud(img)
    X_train[n+3,:,:,:] = fliplr(flipud(img))
    n += 4

# Create the Dice Coefficient and a Dice Loss Function

In [5]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):    
    return (1-dice_coef(y_true, y_pred))

## Create the Models, Initialize with ImageNet Weights

In [6]:
#adamcustom = Adam(lr=0.005)

#import tensorflow_addons as tfa
#custom_optimizer = tfa.optimizers.AdamW(weight_decay=1e-4)
model1 = segmentation_models.Unet('resnet34', encoder_weights='imagenet', input_shape=(int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS), encoder_freeze=True, classes=4, activation='sigmoid')
model2 = segmentation_models.Unet('inceptionv3', encoder_weights='imagenet', input_shape=(int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS), encoder_freeze=True, classes=4, activation='sigmoid')
model3 = segmentation_models.Unet('densenet121', encoder_weights='imagenet', input_shape=(int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS), encoder_freeze=True, classes=4, activation='sigmoid')

In [7]:
model1.compile('Adam', loss=[dice_loss], metrics=[dice_coef])
model2.compile('Adam', loss=[dice_loss], metrics=[dice_coef])
model3.compile('Adam', loss=[dice_loss], metrics=[dice_coef])


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
from sklearn.model_selection import train_test_split
XTraining, XValidation, YTraining, YValidation = train_test_split(X_train,Y_train,test_size=0.1)

In [14]:
XValidation.shape

(380, 128, 800, 3)

## Fit the Models

In [10]:
earlystopper = EarlyStopping(patience=7, verbose=1)
checkpointer = ModelCheckpoint('resnet34_model_shuffled_downsample', verbose=1, save_best_only=True)
model1.fit(XTraining, YTraining, validation_data=(XValidation, YValidation), shuffle=True, batch_size=4, epochs=40,
                    callbacks=[earlystopper,checkpointer])

Train on 3420 samples, validate on 380 samples
Epoch 1/25
3420/3420 [==============================] - 415s - loss: 0.6162 - dice_coef: 0.3838 - val_loss: 0.5005 - val_dice_coef: 0.4995
Epoch 2/25
3420/3420 [==============================] - 377s - loss: 0.4971 - dice_coef: 0.5029 - val_loss: 0.4582 - val_dice_coef: 0.5418
Epoch 3/25
3420/3420 [==============================] - 377s - loss: 0.4627 - dice_coef: 0.5373 - val_loss: 0.4126 - val_dice_coef: 0.5874
Epoch 4/25
3420/3420 [==============================] - 376s - loss: 0.4525 - dice_coef: 0.5475 - val_loss: 0.4086 - val_dice_coef: 0.5914
Epoch 5/25
3420/3420 [==============================] - 376s - loss: 0.4249 - dice_coef: 0.5751 - val_loss: 0.4028 - val_dice_coef: 0.5972
Epoch 6/25
3420/3420 [==============================] - 377s - loss: 0.4121 - dice_coef: 0.5879 - val_loss: 0.3847 - val_dice_coef: 0.6153
Epoch 7/25
3420/3420 [==============================] - 375s - loss: 0.4211 - dice_coef: 0.5789 - val_loss: 0.3869 - va

In [12]:
earlystopper = EarlyStopping(patience=4, verbose=1)
checkpointer = ModelCheckpoint('inception_model_shuffled_downsample', verbose=1, save_best_only=True)
results2 = model2.fit(XTraining, YTraining, validation_data=(XValidation, YValidation), shuffle=True, batch_size=3, epochs=30,
                    callbacks=[earlystopper,checkpointer])

Train on 3800 samples, validate on 380 samples
Epoch 1/30
3800/3800 [==============================] - 794s - loss: 0.6159 - dice_coef: 0.3841 - val_loss: 0.4785 - val_dice_coef: 0.5215
Epoch 2/30
3800/3800 [==============================] - 654s - loss: 0.5219 - dice_coef: 0.4781 - val_loss: 0.4632 - val_dice_coef: 0.5368
Epoch 3/30
3800/3800 [==============================] - 653s - loss: 0.4986 - dice_coef: 0.5014 - val_loss: 0.4296 - val_dice_coef: 0.5704
Epoch 4/30
3800/3800 [==============================] - 654s - loss: 0.4740 - dice_coef: 0.5260 - val_loss: 0.4082 - val_dice_coef: 0.5918
Epoch 5/30
3800/3800 [==============================] - 654s - loss: 0.4560 - dice_coef: 0.5440 - val_loss: 0.4073 - val_dice_coef: 0.5927
Epoch 6/30
3800/3800 [==============================] - 655s - loss: 0.4535 - dice_coef: 0.5465 - val_loss: 0.3847 - val_dice_coef: 0.6153
Epoch 7/30
3800/3800 [==============================] - 653s - loss: 0.4349 - dice_coef: 0.5651 - val_loss: 0.3821 - va

In [10]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('densenet_model_shuffled_downsample', verbose=1, save_best_only=True)
results3 = model3.fit(X_train, Y_train, validation_data=(XValidation, YValidation), shuffle=True, batch_size=2, epochs=40,
                    callbacks=[earlystopper,checkpointer])

Train on 3800 samples, validate on 380 samples
Epoch 1/50
3800/3800 [==============================] - 1581s - loss: 0.7494 - dice_coef: 0.2506 - val_loss: 0.7348 - val_dice_coef: 0.2652
Epoch 2/50
3800/3800 [==============================] - 1417s - loss: 0.6915 - dice_coef: 0.3085 - val_loss: 0.7105 - val_dice_coef: 0.2895
Epoch 3/50
3800/3800 [==============================] - 1406s - loss: 0.6632 - dice_coef: 0.3368 - val_loss: 0.7734 - val_dice_coef: 0.2266
Epoch 4/50
3800/3800 [==============================] - 1408s - loss: 0.6482 - dice_coef: 0.3518 - val_loss: 0.7291 - val_dice_coef: 0.2709
Epoch 5/50
3800/3800 [==============================] - 1408s - loss: 0.6405 - dice_coef: 0.3595 - val_loss: 0.7581 - val_dice_coef: 0.2419
Epoch 6/50
3800/3800 [==============================] - 1409s - loss: 0.6302 - dice_coef: 0.3698 - val_loss: 0.7391 - val_dice_coef: 0.2609
Epoch 7/50
3800/3800 [==============================] - 1410s - loss: 0.6206 - dice_coef: 0.3794 - val_loss: 0.77

In [11]:
del XTraining
del YTraining

## Load the Test Data

In [19]:
testfiles = next(os.walk(TEST_IMAGE_PATH))[2]
X_test = np.zeros((len(testfiles), int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), int(IMG_CHANNELS)), dtype=np.uint8)

n = 0
for file in testfiles:
  
    #img = color.rgb2gray(io.imread(TEST_IMAGE_PATH + file))[:,:]
    img = imread(TEST_IMAGE_PATH + file)
    img = resize(img, (IMG_HEIGHT/SCALE_FACTOR, IMG_WIDTH/SCALE_FACTOR, IMG_CHANNELS), mode='constant', preserve_range=True)
    X_test[n,:,:,:] = img
    if n % 500 == 0:
        print('Loaded test instance', n)
    n += 1

Loaded test instance 0
Loaded test instance 500
Loaded test instance 1000
Loaded test instance 1500
Loaded test instance 2000
Loaded test instance 2500
Loaded test instance 3000
Loaded test instance 3500
Loaded test instance 4000
Loaded test instance 4500
Loaded test instance 5000
Loaded test instance 5500


## Prepare Kaggle Submission

In [20]:
submission_list = []
model1 = load_model('../input/submission-file/resnet34_model', custom_objects={'dice_loss': dice_loss, 'dice_coef': dice_coef})
model2 = load_model(r'C:\Users\Ashley\inception_model_shuffled_downsample', custom_objects={'dice_loss': dice_loss, 'dice_coef': dice_coef})
model3 = load_model('../input/submission-file/densenet_model', custom_objects={'dice_loss': dice_loss, 'dice_coef': dice_coef})
                   
for n in range(len(testfiles)):
    #preds_test1 = model1.predict(X_test[n:n+1,:,:,:])
    preds_test2 = model2.predict(X_test[n:n+1,:,:,:])
    #preds_test3 = model3.predict(X_test[n:n+1,:,:,:])

    for m in range(4):
        #img1 = preds_test1[0,:,:,m]
        img2 = preds_test2[0,:,:,m]
        #img3 = preds_test3[0,:,:,m]
        #resize mask to original size
#         img1 = resize(img, (256, 1600), mode='constant', preserve_range=True)
#         img2 = resize(img, (256, 1600), mode='constant', preserve_range=True)
#         img3 = resize(img, (256, 1600), mode='constant', preserve_range=True)
        #img = np.mean([img1,img2,img3], axis=0)
        img2 = resize(img2, (256, 1600), mode='constant', preserve_range=True)
        img2 = (img2 > 0.5).astype(np.uint8)
        #encode results and put in dataframe
        encoded_entry = mask2rle(img2)
        row = [testfiles[n] + '_' + str(m+1), encoded_entry]
        submission_list.append(row)
    if n % 500 == 0:
        print('Saving entry', n)
    
#create submission file
submission_data = pd.DataFrame(submission_list, columns=['ImageId_ClassId','EncodedPixels'])
submission_data = submission_data.fillna('')
submission_data.to_csv('submission.csv', index=False)

Saving entry 0
Saving entry 500
Saving entry 1000
Saving entry 1500
Saving entry 2000
Saving entry 2500
Saving entry 3000
Saving entry 3500
Saving entry 4000
Saving entry 4500
Saving entry 5000
Saving entry 5500
